In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date

from functions import mins_to_meters, meters_to_mins

from sklearn.linear_model import LinearRegression

# athlete level model

In [2]:
#create model for calculating athlete level
data = {'speed': [4.291573333333333, 5.109015873015872, 2.438393939393939, 2.682233333333333], 
        'distance': [26.2, 3.1, 26.2, 3.1],'level': [10, 10, 1, 1]}
df_pace_lvl = pd.DataFrame(data)

#calculate slope and intercept -linear
X = df_pace_lvl[['speed', 'distance']]
y = df_pace_lvl['level']
level = LinearRegression().fit(X, y)
print('score: ', level.score(X, y))
print('coefficients: ', level.coef_)
print('intercept: ', level.intercept_)

score:  0.9823553977865943
coefficients:  [4.13143795 0.0949052 ]
intercept:  -10.888737461832598


# distance factor model

In [3]:
#modeling distance factor from athl_level (bound to 1-10)
data = {'level': [10, 1], 
        'dist_factor': [1, 0.5]}
df_distance_lvl = pd.DataFrame(data)
dist_model = np.poly1d(np.polyfit(df_distance_lvl['level'], df_distance_lvl['dist_factor'], 1))

# main function for base calendar 

In [4]:
#function to date race date, number of weeks to train, and generate 
def get_calendar(race_year, race_month, race_day, weeks, pace_min, pace_sec, distance):
    
    #calcluate meters per second (speed) from goal pace
    speed = mins_to_meters(m=pace_min, s=pace_sec)
    
    race_date = date(race_year, race_month, race_day+1)
    
    cal = weeks*7
    
    date_list = [race_date - datetime.timedelta(days=x) for x in range(1,(cal+1))]
    
    date_list.reverse()
    
    days = []

    for x in date_list:
        days.append(x.weekday())
        
    data = {'date': date_list, 'day_code': days}
    
    df_training_cal = pd.DataFrame(data)
    
    #create validation table for weekday codes/desc to join to training calendar
    weekdays = {'day_code': range(0,7), 'day_desc': ['Mon', 'Tues', 'Wed', 'Thurs', 'Fri', 'Sat', 'Sun']}
    df_weekdays = pd.DataFrame(weekdays)
    df_weekdays
    
    df_training_cal = pd.merge(df_training_cal, df_weekdays, how='left', on='day_code')
    
    #find first monday and crop calendar down to start on first monday
    first_mon = df_training_cal[df_training_cal.day_code == 0].index[0]
    df_training_cal = df_training_cal.iloc[first_mon:]
    
    #create column for week count
    week = []
    count = 0
    for index, row in df_training_cal.iterrows():
        if row.day_code == 0:
            count += 1
            week.append(count)
        else:
            week.append(count)
    df_training_cal['week']=week
    
    #create column for training phase
    #2 week taper for blocks under 14 weeks, 3 week taper for blocks >= 14 weeks
    block = 0
    if weeks < 14:
        block += (df_training_cal.week.max() - 2)
    if weeks >= 14:
        block += (df_training_cal.week.max() - 3)
    base = np.ceil(block*0.4)
    peak = np.floor(block*0.6)

    phase= []

    for index, row in df_training_cal.iterrows():
        if row.week <= base:
            phase.append('base')
        elif row.week-base <= peak:
            phase.append('peak')
        else:
            phase.append('taper')
    df_training_cal['phase']=phase
    
    #calculate level and assign to level raw (used for pace calc)
    #if level raw is outside range(1,10), bound to nearest level and assign to dist_level (used for max distance calc)
    user_X = pd.DataFrame({'speed': [speed], 'distance': [distance]})
    level_raw = level.predict(user_X)[0]
    dist_level = []
    
    if level_raw < 1:
        dist_level.append(1)
    elif level_raw > 10:
        dist_level.append(10)
    else:
        dist_level.append(level_raw)
        
    #calculate paces (to be used for workouts)
    b1 = level.coef_[0]
    b2 = level.coef_[1]
    b0 = level.intercept_

    five_k = ((level_raw - b0) - (3.1*b2)) / b1
    ten_k = ((level_raw - b0) - (10.2*b2)) / b1
    hmp = ((level_raw - b0) - (13.1*b2)) / b1
    mp = ((level_raw - b0) - (26.2*b2)) / b1
    
    #weekly mileage peak
    mileage_max = 0
    if distance == 3.1:
        mileage_max += 45
    elif distance == 6.2:
        mileage_max += 50
    elif distance == 13.1:
        mileage_max += 60
    elif distance == 26.2:
        mileage_max += 75
        
    dist_factor = dist_model(dist_level[0]) 
    
    user_max = mileage_max*dist_factor
    #user_max = mileage_max-(level_final[0]*3)
    
    #weekly mileage
    base = len(df_training_cal.loc[df_training_cal.phase=='base'].week.unique())
    peak = len(df_training_cal.loc[df_training_cal.phase=='peak'].week.unique())
    taper = len(df_training_cal.loc[df_training_cal.phase=='taper'].week.unique())
    
    week_1 = round(user_max/3, 1)
    build = user_max-week_1
    weekly_miles = []
    
    week_num = []
    
    for index, row in df_training_cal.iterrows():
        if row.week not in week_num:
            week_num.append(row.week)
            
    weekly_miles = [week_1]
    miles = 0
    
    for i in range(1,base):
        miles += build/(base-1)
        weekly_miles.append(round(week_1+miles, 1))
    
    for i in range(1, peak+1):
        weekly_miles.append(round(user_max, 1))
    
    if taper == 2:
        weekly_miles.append(user_max*0.7)
        weekly_miles.append(user_max*0.4)
        
    if taper == 3:
        weekly_miles.append(round(user_max*0.85, 1))
        weekly_miles.append(round(user_max*0.65, 1))
        weekly_miles.append(round(user_max*0.3, 1))
    
    data = {'week': week_num, 'mileage': weekly_miles}
    df_mileage = pd.DataFrame(data)
    
    #add weekly mileage into df_training_cal
    weekly_mileage = []
    for index, row in df_training_cal.iterrows():
        weekly_mileage.append(df_mileage.loc[df_mileage.week == row.week].mileage.values[0])
    
    df_training_cal["weekly_mileage"] = weekly_mileage
    
    #add down week for training blocks >= 14 weeks
    if weeks >= 14:
        down_phase = []
        down_mileage = []
        
        for index, row in df_training_cal.iterrows():
            if row.week == (base + (peak-2)):
                down_phase.append('down')
                down_mileage.append(round(user_max*0.5, 1))
            else:
                down_phase.append(row.phase)
                down_mileage.append(row.weekly_mileage)
                
        df_training_cal['phase']=down_phase
        df_training_cal['weekly_mileage']=down_mileage
    
    return df_training_cal, level_raw, dist_level, user_max, df_mileage, speed, five_k, ten_k, hmp, mp

In [48]:
result = get_calendar(race_year=2023, race_month=11, race_day=19, weeks=16, pace_min=17, pace_sec=5, distance=3.1)
print('raw level: ', result[1])
print('mp: ', meters_to_mins(result[9]))
print('hmp: ', meters_to_mins(result[8]))
print('10k: ', meters_to_mins(result[7]))
print('5k: ', meters_to_mins(result[6]))

raw level:  -4.107684411631659
mp:  (25, 48.24)
hmp:  (20, 0.6599999999999999)
10k:  (19, 3.7800000000000002)
5k:  (17, 4.98)


In [28]:
result[0][['week', 'phase', 'weekly_mileage']].drop_duplicates()

,week,phase,weekly_mileage
0,1,base,10.9
7,2,base,15.2
14,3,base,19.6
21,4,base,23.9
28,5,base,28.3
35,6,base,32.6
42,7,peak,32.6
49,8,peak,32.6
56,9,peak,32.6
63,10,peak,32.6


# pace predict

In [169]:
print('coefficients: ', level.coef_)
print('intercept: ', level.intercept_)
print('level: ', result[1])

coefficients:  [4.13143795 0.0949052 ]
intercept:  -10.888737461832598
level:  9.254081579750807


In [170]:
level_raw = result[1]
b1 = level.coef_[0]
b2 = level.coef_[1]
b0 = level.intercept_

five_k = ((level_raw - b0) - (3.1*b2)) / b1
ten_k = ((level_raw - b0) - (10.2*b2)) / b1
hmp = ((level_raw - b0) - (13.1*b2)) / b1
mp = ((level_raw - b0) - (26.2*b2)) / b1

In [171]:
print('5k pace: ', meters_to_mins(five_k))
print('10k pace: ', meters_to_mins(ten_k))
print('hm pace: ', meters_to_mins(hmp))
print('m pace: ', meters_to_mins(mp))

5k pace:  (5, 34.98)
10k pace:  (5, 46.74)
hm pace:  (5, 51.78)
m pace:  (6, 16.560000000000002)


In [35]:
((3*800 + 3*400)/1000)*0.62

2.232

In [30]:
11*.15

1.65

In [38]:
(1/600)*60

0.1

In [43]:
#jog rest intervals in miles:

print('30 sec: ', (1/600)*30)
print('45 sec: ', (1/600)*45)
print('60 sec: ', (1/600)*60)
print('90 sec: ', (1/600)*90)
print('2 min: ', (1/600)*120)
print('2.5 min: ', (1/600)*150)


30 sec:  0.05
45 sec:  0.07500000000000001
60 sec:  0.1
90 sec:  0.15000000000000002
2 min:  0.2
2.5 min:  0.25


In [42]:
0.2*3 + 0.15*2

0.9000000000000001

In [44]:
0.15 + 0.1*2 + 0.075*4 + 0.05*7

1.0

In [47]:
0.15 + 0.1*2 + 0.075*3

0.575